In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
print(tf.__version__)

2.4.0


# Part 1 - Data Preprocessing

# Importing the dataset

In [3]:
# Importing dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:,3:-1].values # Ignoring row number,customer id & surname since it doesn't have impact on prediction
y = dataset.iloc[:,-1:].values
dataset.shape

(10000, 14)

In [70]:
print(X)
print(y)
print(X[:,2])
print(X[2])

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
[[1]
 [0]
 [1]
 ...
 [1]
 [1]
 [0]]
['Female' 'Female' 'Female' ... 'Female' 'Male' 'Female']
[502 'France' 'Female' 42 8 159660.8 3 1 0 113931.57]


# Encoding Categorial data

# Label encoding of "Gender" column

In [71]:
from sklearn.preprocessing import LabelEncoder
lc = LabelEncoder()
X[:,2] = lc.fit_transform(X[:,2])
lc.fit
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


# One Hot encoding of "Geography" column

In [72]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
# This dummy variable will appear at start of array
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [73]:
#print(f)
#print(ct.named_transformers_['encoder'])
#decoded = ct.named_transformers_['encoder'].inverse_transform(f)
#print(decoded)
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


# Splitting dataset to training and test set

In [81]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)
print(X_train)
print(y_train)

[[0.0 0.0 1.0 ... 1 0 163830.64]
 [0.0 1.0 0.0 ... 1 1 57098.0]
 [1.0 0.0 0.0 ... 1 0 185630.76]
 ...
 [1.0 0.0 0.0 ... 1 0 181429.87]
 [0.0 0.0 1.0 ... 1 1 148750.16]
 [0.0 1.0 0.0 ... 1 0 118855.26]]
[[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [1]]


# Feature Scaling

In [82]:
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)
print(X_train)
print(X_test)

[[-1.01460667 -0.5698444   1.74309049 ...  0.64259497 -1.03227043
   1.10643166]
 [-1.01460667  1.75486502 -0.57369368 ...  0.64259497  0.9687384
  -0.74866447]
 [ 0.98560362 -0.5698444  -0.57369368 ...  0.64259497 -1.03227043
   1.48533467]
 ...
 [ 0.98560362 -0.5698444  -0.57369368 ...  0.64259497 -1.03227043
   1.41231994]
 [-1.01460667 -0.5698444   1.74309049 ...  0.64259497  0.9687384
   0.84432121]
 [-1.01460667  1.75486502 -0.57369368 ...  0.64259497 -1.03227043
   0.32472465]]
[[-1.01460667  1.75486502 -0.57369368 ...  0.64259497  0.9687384
   1.61085707]
 [ 0.98560362 -0.5698444  -0.57369368 ...  0.64259497 -1.03227043
   0.49587037]
 [-1.01460667 -0.5698444   1.74309049 ...  0.64259497  0.9687384
  -0.42478674]
 ...
 [-1.01460667 -0.5698444   1.74309049 ...  0.64259497 -1.03227043
   0.71888467]
 [-1.01460667  1.75486502 -0.57369368 ...  0.64259497  0.9687384
  -1.54507805]
 [-1.01460667  1.75486502 -0.57369368 ...  0.64259497 -1.03227043
   1.61255917]]


# Part 2: Building ANN

# Intializing ANN

In [83]:
ann = tf.keras.models.Sequential()

# Adding Input layer and first hidden layer

In [84]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu')) #units- No.of Neuron, relu-Rectifier activation function

# Adding second hidden layer

In [85]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu')) #units- No.of Neuron, relu-Rectified Linear Unit activation function

# Adding output layer

In [86]:
# unit is 1 since we have output as(0 or 1). If dependent variable output is A,B&C then we need 3 neuron
# activation is set to Sigmoid, it returns probability(0 to 1) since dependent variable in our case is 0 or 1 
# For non binary o/p(more than 2 category -A,B,C etc), activation should be 'softmax'
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Part 3:Training ANN

# Compiling ANN

Updating weight of each neuron in hidden layer to reduce the loss over the epoch. Signal from Feature or dependent variable is forward propagated to hidden layers and then to output layer. Predicted result is compared to actual result in Energy output. This incur the loss which is exactly squared difference between predicted vs actual energy output. This predictions comes in batches which will sum squared difference between predicted vs actual energy output, incurs loss. So, backward propogation is applied to neural network. Adam Stochastic gradient descent is applied with optimiser to reduce the loss.

In [87]:
ann.compile(optimizer='adam' , loss= 'binary_crossentropy', metrics= ['accuracy']) 
# If the output isn't binary(0 or 1) then loss will be categorical_crossentropy

# Training ANN on the training set

In [88]:
ann.fit(X_train, y_train, batch_size=32, epochs=100) #epochs - Iteraction

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.5345 - accuracy: 0.7907
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4576 - accuracy: 0.7971
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4450 - accuracy: 0.7960
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4290 - accuracy: 0.8005
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4241 - accuracy: 0.8033
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4180 - accuracy: 0.8171
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4050 - accuracy: 0.8190
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4039 - accuracy: 0.8221
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3846 - accuracy: 0.8328
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3867 - accura

# Part 4:Making Prediction and evaluating the model

# Predicting result of single observation

 **Homework**
 
 Use our ANN model to predict if the customer with the following informations will leave the bank: 
 
 Geography: France
 
 Credit Score: 600
 
 Gender: Male
 
 Age: 40 years old
 
 Tenure: 3 years
 
 Balance: \$ 60000
 
 Number of Products: 2
 
 Does this customer have a credit card? Yes
 
 Is this customer an Active Member: Yes
 
 Estimated Salary: \$ 50000
 
 So, should we say goodbye to that customer?

In [91]:
#If predicted probability is grater than 0.5 then 1 else 0
print(ann.predict(ss.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

[[False]]


 Therefore, our ANN model predicts that this customer stays in the bank!
 
 **Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.
 
 **Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.


# Predicting test set result

In [92]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5) # 0 if y_pred < 0.5 else 1
# 0 - stayed in the bank & 1 - left the bank
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


# Making confusion matrix

In [119]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report
cm = confusion_matrix(y_test,y_pred)
print(cm)

tp, fn, fp, tn = cm.reshape(-1)
print("tp, fn, fp, tn : ", tp, fn, fp, tn)

print('Accuracy : ', accuracy_score(y_test,y_pred))

print('Precision : ', precision_score(y_test,y_pred, average=None))
print('Weighted Precision : ', precision_score(y_test,y_pred, average='weighted'))
print('macro Precision : ', precision_score(y_test,y_pred, average='macro'))
print('Micro Precision : ', precision_score(y_test,y_pred, average='macro'))

print('Recall : ', recall_score(y_test,y_pred, average=None))
print('F1 Score : ', f1_score(y_test,y_pred, average=None))

print('*********Manual Calculation***********')
print('Accuracy : ', (tp+tn)/(tp+fn+fp+tn))
precision_0 = tp/(tp+fp)
precision_1 = tn/(tn+fn)
print('Precision_0 : ',precision_0 ,', precision_1', precision_1)
print('Precision : ', (precision_0+precision_1)/2)

print('recall_0 : ', tp/(tp+fn))
print('recall_1 : ', tn/(tn+fp))

print('************ classification report *****************')
cp = classification_report(y_test, y_pred)
print(cp)

[[1500   95]
 [ 198  207]]
tp, fn, fp, tn :  1500 95 198 207
Accuracy :  0.8535
Precision :  [0.88339223 0.68543046]
Weighted Precision :  0.843304969227529
macro Precision :  0.7844113448622844
Micro Precision :  0.7844113448622844
Recall :  [0.94043887 0.51111111]
F1 Score :  [0.91102338 0.58557284]
*********Manual Calculation***********
Accuracy :  0.8535
Precision_0 :  0.8833922261484098 , precision_1 0.6854304635761589
Precision :  0.7844113448622844
recall_0 :  0.9404388714733543
recall_1 :  0.5111111111111111
************ classification report *****************
              precision    recall  f1-score   support

           0       0.88      0.94      0.91      1595
           1       0.69      0.51      0.59       405

    accuracy                           0.85      2000
   macro avg       0.78      0.73      0.75      2000
weighted avg       0.84      0.85      0.85      2000

